In [ ]:
import xarray as xr
from os.path import join
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

## read and align data

In [ ]:
# data directory
ddir = r'../../1_data/2_forcing'

# data labels
labels = {
    'qb': 'Discharge Buzi\n[m3/s]',
    'qp': 'Discharge Pungwe\n[m3/s]',
    'p': 'Rainfall\n[mm/hr]',
    't': 'Tide\n[m+MSL]',
    's': 'Surge\n[m]',
    'w': 'Sign. wave height\n[m]',
    'h_ts': 'Total waterlevel\n[m+MSL]',
    'h_tsw': 'Total waterlevel (incl. wave setup)\n[m+MSL]',
    'ss': 'Skew surge\n[m]',
    'ssw': 'Skew surge (incl. wave setup)\n[m]',
    'sw': 'Non-tidal residual\n[m]'
}

In [ ]:
import hydromt
from eva import eva_block_maxima, get_peaks, get_peak_hydrographs

# discharge
fnq = join(ddir, 'cama_discharge_beira_daily.nc')
daq = xr.open_dataset(fnq)['discharge'].load()
dsq = xr.merge([
    daq.sel(index=1).rename('qb').reset_coords(drop=True),
    daq.sel(index=4).rename('qp').reset_coords(drop=True)
])
# fnq = r'../../3_models/wflow/run_vito_ksath1000/output_src.nc'
# daq = xr.open_dataset(fnq)['q_river'].load()
# dsq = xr.merge([
#     daq.sel(index=0).rename('qb').reset_coords(drop=True),
#     daq.sel(index=3).rename('qp').reset_coords(drop=True)
# ])

# save qbankfull at river inflow locations of SFINCS
dsq_eva = eva_block_maxima(
    daq, period = 'AS-AUG', min_dist = 14,
)
gdf_qbf = dsq_eva['return_values'].sel(rps=2).to_dataset().vector.to_gdf().rename(columns={'return_values': 'qbankfull'})
gdf_qbf.to_file(fnq.replace('.nc', '_qbf.geojson'), driver='GeoJSON')
gdf_qbf['qbankfull']

In [ ]:
# GTSM waterlevels + ERA5 waves
# contains: "waterlevel" (tide+surge), "tide", "surge", "shww"
fnh = join(ddir, 'reanalysis_gtsm_v1_beira_extended.nc')
dsh0 = xr.open_dataset(fnh).load()
dsh0 = dsh0.rename({'waterlevel': 'h_ts', 'surge': 's', 'tide': 't', 'shww': 'w'})
dsh0['h_tsw'] = dsh0['h_ts'] + 0.2*dsh0['w']
dsh0['sw'] = dsh0['s'].fillna(0) + 0.2*dsh0['w']
# skew surge (not used)
# high_tide = get_peaks(dsh0['t'].load(), period='12H').dropna('time').reindex_like(dsh0, 'nearest')
# dsh0['ss'] = dsh0['h_ts'] - high_tide
# dsh0['ssw'] = dsh0['h_tsw'] - high_tide

In [ ]:
from eva import eva_idf, get_hyetograph
# ERA5 precipitation
fnp = join(ddir, 'era5_precip_beira_hourly_spatialmean.nc')
dap0 = xr.open_dataset(fnp, chunks='auto')['precip'].load()

In [ ]:
# resample to day
dates = pd.date_range('19800102', '20210201', freq='D')
# dap = dap0.resample(time='1D', label='right').sum('time')
# dsh = dsh0.resample(time='1D', label='right').max('time')
dap = dap0.rolling(time=24, center=True, min_periods=1).mean('time').reindex(time=dates)
dsh = dsh0.rolling(time=6*24, center=True, min_periods=1).max('time').reindex(time=dates)

# merge all variables to singlge dataset
ds = xr.merge([
    dsq.reindex(time=dates),
    dsh,
    dap.rename('p')
], compat='override').reset_coords(drop=True)#.reindex(time=dates)
for var in ds.data_vars:
    long_name, unit = labels[var].split('\n')
    ds[var].attrs.update({'long_name': long_name, 'unit': unit[1:-1]})
ds.attrs = {}
encoding = {var: {'zlib': True} for var in ds.data_vars}
# ds.to_netcdf(join(ddir, 'beira_drivers_daily.nc'), encoding=encoding)
ds = ds[['qb', 'qp', 'p', 'h_tsw', 't', 's', 'w']]

In [ ]:
# ds = xr.open_dataset(join(ddir, 'beira_drivers_daily.nc'))
ds = ds[['qb', 'qp', 'p', 'h_tsw', 't', 's', 'w']]

## get annual maxima peaks

In [ ]:
from eva import get_peak_hydrographs, get_peaks

# settings
period='AS-AUG'

ds_peaks = xr.Dataset(coords=ds.coords)
for dvar in ds.data_vars.keys():
    if dvar == 't': continue
    ds_peaks[dvar] = get_peaks(ds[dvar], period=period, min_dist=14, min_sample_size=0)

# peaks with dates
df_peaks0 = ds_peaks.reset_coords(drop=True).dropna('time', how='all').to_dataframe()  

# get maximum values within time window for non_extremes
df_peaks0_filled = pd.DataFrame()
ds_tmax = ds.rolling(time=7).max('time').sel(time=df_peaks0.index)
# ds_tmax = ds.sel(time=df_peaks0.index)
for dvar in df_peaks0.columns:
    df_peaks0_filled[dvar] = df_peaks0[dvar].where(df_peaks0[dvar].notna(), ds_tmax[dvar])

# peaks with regular spaced interval
df_bm = ds_peaks.resample(time=period).max('time').to_dataframe().dropna()


In [ ]:
n = len(ds_peaks.data_vars)
fig, axes = plt.subplots(n, 1, figsize=(12, 3*n), sharex=True)
for i, dvar in enumerate(ds_peaks.data_vars.keys()):
    ds[dvar].to_series().plot(ax=axes[i], color='k')
    df_peaks0[dvar].plot(ax=axes[i], color='r', marker='.', lw=0)
    axes[i].set_ylabel(labels[dvar])

## fit uni-variate eva

In [ ]:
from scipy.interpolate import interp1d
from scipy.stats import gaussian_kde, rankdata

class emperical_dist(object):
    def __init__(self, data, nyears):
        self.data = np.sort(data)[::-1]  # descending
        nevents = data.size
        self.freq = np.arange(1,nevents+1)/(nevents+1)*(nevents/nyears)
        self.data = self.data[self.freq<1]
        self.freq = self.freq[self.freq<1]

    def ppf(self, q):
        return interp1d(
            1-self.freq, 
            self.data, 
            bounds_error = False, 
            fill_value=(self.data[-1], self.data[0])
        )(q)
        
    def pdf(self, x, **kwargs):
        return gaussian_kde(self.data, **kwargs)(x)

    def sf(self, x):
        return interp1d(
            self.data,
            self.freq,
            bounds_error = False, 
            fill_value=(self.freq[-1], self.freq[0])
        )(x)

class rps_dist(object):
    def __init__(self, rps, rvs):
        self.rps = rps
        self.rvs = rvs

    def ppf(self, q):
        return interp1d(
            1-1/self.rps,
            self.rvs,
            bounds_error = False, 
            fill_value=(self.rvs[-1], self.rvs[0])
        )(q)

In [ ]:
from eva import lmoment_fitopt, get_frozen_dist, _get_return_periods, _RPS

# prepare surge extremes
fileS = 'Beira_STORM_surges.nc' #We look at the 3000 yr of data
da_surge = xr.open_dataarray(join(ddir, fileS))#[-3000:]

x_etc = ds_peaks['s'].dropna('time').to_series().sort_values().values[:-2]  # filter 2 TCs
params, dist = lmoment_fitopt(x_etc, distributions=['gev', 'gumb'], criterium='AIC')
dist_etc = get_frozen_dist(params, dist)

x_tc = da_surge.values.flatten()
dist_tc = emperical_dist(x_tc, 3000)

# combine: rp(x) = 1 / (1/rp(x_TC) + 1/rp(x_ETC))
xs = np.arange(0.1, np.max(x_tc), 0.1)
rp_tc = 1/dist_tc.sf(xs)
rp_etc = 1/(1-dist_etc.cdf(xs))
rp_tot = 1/(1/rp_etc + 1/rp_tc)
dist_surge = rps_dist(rp_tot, xs)

# plot
fgumbplot = lambda x: -np.log(-np.log(1.0 - 1.0 / x))

fig, ax = plt.subplots(1,1)
ax.plot(fgumbplot(_get_return_periods(x_etc)), x_etc, '.b', label='ETC')
ax.plot(fgumbplot(_get_return_periods(x_tc, extremes_rate=x_tc.size/3000)), x_tc, '.r', label='TC')
ax.plot(fgumbplot(rp_tot), xs, '--k', lw=2, label='combined')
ax.set_ylabel("Return value")
ax.set_xticks(fgumbplot(_RPS))
ax.set_xticklabels(_RPS)
ax.set_xlabel("Return period")
ax.set_xlim([fgumbplot(1.1), fgumbplot(1000)])
ax.grid()
ax.legend()


In [ ]:
from eva import lmoment_fitopt, get_frozen_dist, plot_return_values, _RPS, _get_return_values
fgumbplot = lambda x: -np.log(-np.log(1.0 - 1.0 / x))

n = len(ds_peaks.data_vars)
dparams = ['shape', 'loc', 'scale']
distributions = ['gev', 'gumb']#[1:]

fig, axes = plt.subplots(n, 1, figsize=(8, 3*n), sharex=True)

df_eva = pd.DataFrame(columns=['dist'] + dparams)
df_rps = pd.DataFrame(index=np.hstack([[1.1], _RPS]))
df_rps.index.name = 'rps'
dists = {}

# use marginal distributions to transform quantiles back to normal space
for dvar in df_eva.index:
    params = df_eva.loc[dvar, dparams].dropna()
    dist = df_eva.loc[dvar, 'dist']

for i, dvar in enumerate(ds_peaks.data_vars.keys()):

    if dvar == 's':
        dists[dvar] = dist_surge
        df_rps[dvar] = dist_surge.ppf(1-1/df_rps.index.values)
        axes[i].plot(fgumbplot(_get_return_periods(x_etc)), x_etc, '.b', label='ETC')
        axes[i].plot(fgumbplot(_get_return_periods(x_tc, extremes_rate=x_tc.size/3000)), x_tc, '.r', label='TC')
        axes[i].plot(fgumbplot(rp_tot), xs, '--k', lw=2, label='combined')
        axes[i].set_ylabel("Return value")
        axes[i].set_xticks(fgumbplot(_RPS))
        axes[i].set_xticklabels(_RPS)
        axes[i].set_xlabel("Return period")
        axes[i].grid()        
        axes[i].legend()
    else:
        if dvar == 'p':
            durations=np.array([1, 2, 3, 6, 12, 24], dtype=int)
            da_p_bm = eva_idf(dap0, durations=durations, distribution='gumb', rps=df_rps.index.values)
            da_p_bm0 = da_p_bm.sel(duration=24)
            x = da_p_bm0['peaks'].dropna('time').values
            params = da_p_bm0['parameters'].values[1:]
            dist = da_p_bm0['distribution'].item()
        else:
            x = ds_peaks[dvar].dropna('time').values
            params, dist = lmoment_fitopt(x, distributions=distributions, criterium='AIC')
        dists[dvar] = get_frozen_dist(params, dist)
        df_eva.loc[dvar, dparams[-len(params):]] = params
        df_eva.loc[dvar, 'dist'] = dist
        df_rps[dvar] = _get_return_values(params, dist, rps=df_rps.index.values)
        _ = plot_return_values(x, params, dist, ax=axes[i])
        axes[i].set_ylim([x.min()*0.9, axes[i].get_ylim()[1]])

    axes[i].set_xlim([0.01, fgumbplot(1000)])
    axes[i].set_title(dvar)
    
    axes[i].set_ylabel(labels[dvar])
    if i < n-1:
        axes[i].set_xlabel('')

In [ ]:
# parameters
df_eva

## analyse lag-times

In [ ]:
from datetime import timedelta
from hydromt.stats import pearson_correlation 

def time_lag_crosscorr(
    sim, obs, quantile=None, lags=np.arange(-10,11,1), t_unit='days', dim='time'
):
    """Returns the time lag between two time series based on a lag time 
    with the maximum pearson correlation.
    
    Parameters
    ----------
    sim : xarray DataArray
        simulations time series
    obs : xarray DataArray
        observations time series
    quantile : numpy ndarray, optional
        quantile based threshold (the default is None, which does not use any threshold)
    lags : numpy ndarray, optional
        range of considered lag times (the default is np.arange(-10,11,1))
    t_unit : str, optional
        time unit used to parse lags to timedelta format (the default is 'days')
    dim : str, optional
        name of time dimension in sim and obs (the default is 'time')
    
    Returns
    -------
    xarray DataSet
        lag time and associated correlation coefficient
    """

    if quantile:
        obs.load()        
        obs = obs.where(obs>=obs.quantile(quantile, dim=dim))
    # loop through time lags and calculate cross correlation
    r = []
    lags = np.asarray(lags)
    time_org = sim[dim].to_index()
    for dt in lags:
        time_new = time_org + timedelta(**{t_unit: float(dt)})
        ts = slice(max(time_org.min(), time_new.min()), min(time_org.max(), time_new.max()))
        sim[dim] = time_new
        r.append(pearson_correlation(sim.sel(**{dim:ts}), obs.sel(**{dim:ts})))
    sim[dim] = time_org # reset time
    pearsonr = xr.concat(r, dim='dt')
    pearsonr['dt'] = xr.Variable('dt', lags)
    # get maximum cross corr
    pearsonr_max = pearsonr.max(dim='dt')
    pearsonr_max.name = 'lag_rho'
    pearsonr_max.attrs.update(description='maximum pearson coefficient for given time lag')
    # get lag time of maximum cross corr
    # NOTE that we assume a evenly spaced lag times
    lag = xr.where(
        xr.ufuncs.isfinite(pearsonr).sum(dim='dt')==lags.size,
        pearsonr.argmax(dim='dt', skipna=False), 
        np.nan)*np.diff(lags)[0] + lags.min()
    lag.name = 'lag'
    lag.attrs.update(description='time lag with maximum pearson coefficient', unit=t_unit)
    # merge max cross corr and lag tiem
    return xr.merge([lag, pearsonr_max])

In [ ]:
ref_dvar = 'qb'
dt_lst =[]
dvar_lst = ['qp', 'p', 's', 'w']
for dvar in dvar_lst:
    da_out = time_lag_crosscorr(ds[ref_dvar], ds[dvar]).reset_coords(drop=True).compute()
    dt_lst.append(da_out)
ds_dt = xr.concat(dt_lst, dim='dvar')
ds_dt['dvar'] = xr.IndexVariable('dvar', dvar_lst)
df_timelags = ds_dt['lag'].to_series()
# df_timelags.to_csv(r'../../4_results/lagtimes.csv')
df_timelags

## analyse co-occurence

In [ ]:
# NOTE that `max_time_lag` is the timelag between two consecutive extremes
# the total time lag between all four events could theoretically be the sum of all lagtime per driver
## settings
drivers = [v for v in ['qb', 'qp', 'p', 's', 'w'] if v in ds]
max_time_lag = {
    'qp': pd.Timedelta('5D'),
    'qb': pd.Timedelta('5D'),
}
dt0 = pd.Timedelta('2D')  # default time lag

# reduce daily events to selected drivers
df_peaks = df_peaks0[drivers].dropna(how='all')

# combine daily events to event sets based on maximum time_lag
df_peaks['dt'] = np.hstack([[0], np.diff(df_peaks.index)])
df_peaks['max_dt'] = (df_peaks[drivers].notna()*np.array([max_time_lag.get(d,dt0) for d in drivers])).max(axis=1)
# df_peaks['max_dt'] = [pd.Timedelta(days=d) for d in df_peaks['max_dt'].dt.days.rolling(2, min_periods=1).max().values]
df_peaks['event'] = np.cumsum(~(df_peaks['dt']<df_peaks['max_dt']))
df_events = df_peaks.groupby('event').apply(lambda x: x.notna().sum()).drop(columns=['dt', 'event'])
df_events['n_extreme'] = df_events[drivers].sum(axis=1)
df_events['time'] = df_peaks.reset_index().groupby('event').first()['time']
offset = pd.Timedelta(days=df_bm.index.dayofyear.min()-1)
df_events['year'] = (df_events['time'] - offset).dt.year
df_events['time_lag'] = [evnt.iloc[1:,:]['dt'].sum() for _, evnt in df_peaks.groupby('event')]
evnts, cnts = np.unique(df_peaks['event'], return_counts=True)
df_peaks['co-occur'] = np.isin(df_peaks['event'], evnts[cnts>1])

# make barcode plot for co-occuring events
data = df_events[drivers[::-1]].T * df_events['n_extreme']
_, ax = plt.subplots(1,1, figsize=(12,5))
im = ax.imshow(data.where(data>0), interpolation='none', cmap='Blues', vmin=0, vmax=len(drivers))
ax.set_aspect(2)
ax.set_yticks(np.arange(len(drivers)))
ax.set_yticklabels([labels[dvar].split('\n')[0] for dvar in drivers[::-1]])
# fig.colorbar(im, orientation='vertical')
ax.set_xlabel('events timeline')
xlabs = df_events.reset_index().groupby('year').first()[['event']]
_ = ax.set_xticks(xlabs['event'].values[::2]-1.5)
_ = ax.set_xticklabels(xlabs.index.values[::2], rotation=45)

plt.tight_layout()
# plt.savefig(join(r'../../4_results', 'co-occurence_time.png'), dpi=300, bbox_axes='tight')


# remove incomplete years
check = df_events[['n_extreme', 'year']].groupby('year').sum()
drop_years = check[check['n_extreme']!=len(drivers)].index.values
if drop_years.size > 1:
    df_events = df_events[~np.isin(df_events['year'], drop_years)]
    print(f'ignore years with more/less events than drivers: {drop_years}')
print(f'no. of valid years: {np.unique(df_events.year).size}')
print(f'no. of events: {df_events.index.size}; (>1 extreme: {df_events[df_events.n_extreme>1].index.size})')
print(f'max time lag: {df_events.time_lag.max().days} days')



In [ ]:
df_events_dist = df_events.groupby(drivers + ['n_extreme']).size().reset_index(name='count')
df_events_dist = df_events_dist.sort_values('count', ascending=False)


fig, (ax1, ax) = plt.subplots(2,1, figsize=(7,3), sharex=True, gridspec_kw={'height_ratios': [2, 1], 'hspace':0.0})
df_events_dist.reset_index()['count'].plot.bar(ax=ax1, color='k')
ax1.set_ylabel('count [-]')

x, y = np.where(df_events_dist[drivers])
ax.scatter(x, y, color='k')
ax.set_ylim([-0.5, len(drivers)-0.5])
ax.set_yticks(np.arange(len(drivers)))
ax.set_yticklabels([labels[dvar].split('\n')[0] for dvar in drivers])
ax.set_xlabel('event type')
ax.set_xticklabels('')

plt.tight_layout()

# plt.savefig(join(r'../../4_results', 'co-occurence_hist.png'), dpi=300, bbox_axes='tight')

## dependence modelling

In [ ]:
import pyvinecopulib as pv

# Transform copula data using the empirical distribution
df_uobs = pd.DataFrame(columns=drivers, data = pv.to_pseudo_obs(df_bm[drivers].values))

# fit copula
controls = pv.FitControlsVinecop(
    # family_set=[pv.BicopFamily(x) for x in np.arange(0,11)],
    family_set=[pv.BicopFamily(0)], # idependent only
    # parametric_method='itau',
    # nonparametric_method='quadratic'
    threshold=0.05, # tau threshold   (0.0 default!)
    selection_criterion='aic', # loglik, bic, aic (bic default!)
    # tree_criterion='tau',  # tau, hoeffd, rho, mcor -> no effect ?!
    show_trace=True,
)
cop = pv.Vinecop(data=df_uobs.values, controls=controls)#, structure=pv.RVineStructure(len(drivers)))
# cop.structure

In [ ]:
data = []
m = cop.matrix
n = m.shape[0]
v = drivers
for t in range(n-1):
    # print(f'** Tree: {t:d}')
    for e in range(n-1-t):
        p1, p2 = v[int(m[n-1-e,e]-1)], v[int(m[t,e]-1)]
        px = [v[int(p-1)] for p in m[:t,e]]
        c = cop.get_pair_copula(t,e)
        tau = cop.get_tau(t,e)  # NOTE: diffferent from scipy.stats method ?
        pxs = f' | ' + ','.join(px) if px else ''
        cstr = c.str().replace(f'\n',',')
        print(f'{p1},{p2}{pxs}: {cstr}; tau = {tau:.5f}')
        data.append([t, e, [p1,p2], px, c.str().split(',')[0], c.parameters.flatten(), tau])

df_cop = pd.DataFrame(
    data=data,
    columns=['tree', 'edge', 'pair', 'conditional', 'copula', 'parameters', 'tau']
)
df_cop.set_index(['tree', 'edge'])


In [ ]:
# Sample from the copula
seeds = np.arange(len(drivers), dtype=int)
n_sim = 10000
df_usim = pd.DataFrame(data=cop.simulate(n_sim, seeds=seeds), columns=drivers)

# transform back based on marginal distributions
df_sim = pd.DataFrame()
s = df_usim.index.size
for dvar in drivers:
    df_sim[dvar] = np.maximum(0, dists[dvar].ppf(df_usim[dvar].values))

# df_sim_q = pd.DataFrame()
# df_sim_q[dvar] = np.quantile(df_bm[dvar], df_usim[dvar])
    
df_sim.head()

In [ ]:
from scipy.stats import kendalltau, pearsonr, gaussian_kde
n = len(drivers)-1
fig, axes = plt.subplots(n,n, sharex=False, sharey=False, figsize=(n*2.5,n*2.5), gridspec_kw={'hspace':0.0, 'wspace':0.0})

# df_uobs_cooc = df_uobs.where(df_peaks[drivers][df_peaks['co-occur']].resample(period).max().reset_index(drop=True).notna())
df_obs_cooc = df_peaks[drivers][df_peaks['co-occur']].resample(period).max()

for r in range(n):
    for c in range(n):
        if c > r:
            axes[r,c].set_visible(False)
            continue

for _, c0 in df_cop.iterrows():
    c = c0.edge 
    r = c + c0.tree
    ax = axes[r,c]
    xlab, ylab = c0.pair
    x, y = df_bm[xlab].values, df_bm[ylab].values
    xmin, xmax = np.quantile(x, [0,0.95])
    ymin, ymax = np.quantile(y, [0,0.95])
    ymax = ymax + (ymax-ymin)*0.4
    
    # if len(c0) > 0 and c0.copula != 'Independence':
    xx, yy = np.mgrid[xmin:xmax:100j, ymin:ymax:100j]
    positions = np.vstack([xx.ravel(), yy.ravel()])
    # kernel = gaussian_kde(df_bm[[xlab,ylab]].values.T, bw_method=0.5)
    kernel = gaussian_kde(df_sim[[xlab,ylab]].values.T, bw_method=0.5)
    f = np.reshape(kernel(positions).T, xx.shape)
    cmap = 'Blues' if  c0.copula != 'Independence' else 'Greens'
    ax.contourf(xx, yy, f, cmap=cmap, alpha=0.5)

    # plot events
    x, y = df_bm[xlab].values, df_bm[ylab].values
    xcooc, ycooc = df_obs_cooc[xlab].values, df_obs_cooc[ylab].values
    ax.scatter(xcooc, ycooc, s=13, color='darkorange', label='compound', zorder=2)
    ax.scatter(x, y, s=12, color='black', label='other', alpha=0.9, zorder=1)
    if r == n-1 and c == n-2:
        ax.legend(title='AM events', loc='lower left', bbox_to_anchor=(1.1, 1.1))
    kwargs = dict(        
        horizontalalignment='left',
        verticalalignment='top',
        transform=ax.transAxes,
        bbox=dict(facecolor='white', alpha=0.5, lw=0.1)
    )
    # tau, ptau = kendalltau(x, y)
    # rho, prho = pearsonr(x, y)
    # tsig = ('**' if ptau < 0.05 else '*') if ptau < 0.10 else ''
    # rsig = ('**' if prho < 0.05 else '*') if prho < 0.10 else ''
    # name = c0.copula if c0.copula != 'Independence' else 'Indep.'
    cs = ' | ' + ','.join(c0.conditional) if c0.tree > 0 else ''
    ps = ','.join(c0.pair[::-1])
    ns = f'{c0.copula}' + r' ($\tau$=' + f'{c0.tau:.2f})' if c0.copula != 'Independence' else c0.copula
    txt = f'[{ps}{cs}]\n{ns}' #{tsig}; ' + r'$\rho$='+f'{rho:.2f}{rsig})'
    ax.text(0.02, 0.98, txt, **kwargs)

    if c == 0:
        ax.set_ylabel(labels[ylab].replace('\n', f' ({ylab})\n'))
    else:
        ax.set_yticklabels('')
    if r == n-1:
        ax.set_xlabel(labels[xlab].replace('\n', f' ({xlab})\n'))
    ax.set_ylim([ymin, ymax])
    ax.set_xlim([xmin, xmax])
    
# plt.savefig(join(r'../../4_results', 'copula_autofit.png'), dpi=300, bbox_axes='tight')

## create stochastic event set

In [ ]:
# combine co-occurence and simulations
from itertools import cycle
event_cycle = cycle(df_events.groupby('year'))
sim_events = []
for year, row in df_sim[drivers].iterrows():
    event_yr = next(event_cycle)[1][drivers] 
    events = (event_yr * row)
    events['year'] = year+1
    sim_events.append(events)

df_sim_events0 = pd.concat(sim_events, axis=0, ignore_index=True)
df_sim_events = df_sim_events0
df_sim_events0.head()

In [ ]:
# required for simulating h_tws & plot comparing sim with obs events
xval_norm = {}
thresh_lst = df_sim.quantile(0.01)
s = df_sim_events0.index.size
df_sim_events = df_sim_events0.copy(deep=True)
for dvar in drivers:
    # we randomly sample non-extremes from all values below the rp1 threshold
    thresh = dists[dvar].ppf(1-1/1.01)
    xval_norm = ds[dvar].where(ds[dvar] < thresh,drop=True).to_series().dropna()
    x0 = xval_norm.sample(s, replace=True, random_state=np.sum(seeds)) 
    xs = df_sim_events0[dvar]
    df_sim_events[dvar] = np.where(xs==0, x0, xs)

In [ ]:
# combine waterlevel components
tide = ds['t'].dropna('time').to_series()  # daily high tide
tide_am = ds['t'].resample(time='AS').max('time').dropna('time').to_series()
# combine surge with random daily high tide
df_sim_events['t'] = tide.sample(s, replace=True, random_state=np.sum(seeds)).values

hname = 'h_tsw' if 'w' in ds else 'h_ts' 
df_sim_events[hname] = df_sim_events['t'] + df_sim_events['s'] 
if 'w' in ds:
    df_sim_events[hname] = df_sim_events[hname] + 0.2 * df_sim_events['w']
# at least AM tide for events with extreme
htot_am_idx = df_sim_events[[hname, 'year']].groupby('year').idxmax().values.flatten()
h_am = np.maximum(df_sim_events[hname], tide_am.sample(s, replace=True, random_state=np.sum(seeds)).values)
df_sim_events.loc[htot_am_idx, hname] = np.maximum(df_sim_events.loc[htot_am_idx, hname], h_am.loc[htot_am_idx])

# get AM waterlevel
df_sim_am = df_sim_events.groupby('year').max()
df_sim_am.head()

In [ ]:
# save results important to keep return values of h_tsw consistent with sensitivity analysis
df_sim_am.round(3).to_csv('../../4_results/sim_AM_indep.csv')
df_sim_events.round(3).to_csv('../../4_results/sim_EVENTS_indep.csv')

In [ ]:
dvars = drivers + [hname]
rm = {k:v for k,v in labels.items() if k in dvars}
rm_lab = {k: v.replace(' (','\n(') for k,v in rm.items()}
df_obs_events = df_peaks0_filled[dvars].dropna(how='any')
df_merged = pd.concat([
    df_sim_events[dvars],
    df_obs_events, 
    ], ignore_index=True, axis=0).rename(columns=rm)
colors = np.hstack([
    np.full(df_sim_events.index.size, "k"),
    np.full(df_obs_events.index.size, "r"), 
    ])
sizes = np.hstack([
    np.full(df_sim_events.index.size, 5),
    np.full(df_obs_events.index.size, 30), 
    ])


In [ ]:
axes = pd.plotting.scatter_matrix(df_merged, color=colors, s=sizes, alpha=1.0, diagonal='kde', figsize=(10,10))
for i, dvar in enumerate(dvars):
    axes[i,i].lines.pop(0)
    xmin, xmax = df_merged[rm[dvar]].quantile([0.0, 0.998]).values
    rp1 = dists[dvar].ppf(1-1/1.01)
    # rp1 = thresh_lst[dvar]
    for j in range(len(dvars)):
        axes[j,i].set_xlim([xmin, xmax])
        axes[j,i].axvline(rp1, color='c', ls='-', label='rp1')#, lw=0.5)
        if j != i: 
            axes[i,j].set_ylim([xmin, xmax])
            axes[i,j].axhline(rp1, color='c', ls='--')#, lw=0.5)
    # fix ticks ..
    xticks = axes[-1,i].get_xticks()
    xticks = xticks[np.logical_and(xticks>xmin, xticks<xmax)]
    axes[-1,i].set_xticks(xticks)
    axes[-1,i].set_xticklabels(xticks, rotation=0)
    if i == 0:
        ymin,ymax = axes[0,0].get_ylim()
        axes[i,i].set_yticks(xticks*(ymax-ymin)/(xmax-xmin))
        axes[i,i].set_yticklabels(xticks)
    df_sim_events[dvar].plot.kde(ax=axes[i,i], color='k', label='sim', legend=i==0)
    df_obs_events[dvar].plot.kde(ax=axes[i,i], color='r', label='obs', legend=i==0)
    if i == 0:
        axes[i,i].set_ylabel(rm[dvar])
    else:
        axes[i,i].yaxis.set_visible(False)

for r in range(len(dvars)):
    for c in range(len(dvars)):
        if c > r:
            axes[r,c].set_visible(False)
            continue

In [ ]:
df_merged = pd.concat([
    df_sim_am[dvars],
    df_bm[dvars], 
    ], ignore_index=True, axis=0).rename(columns=rm_lab)
colors = np.hstack([
    np.full(df_sim_am.index.size, "k"),
    np.full(df_bm.index.size, "r"), 
    ])
sizes = np.hstack([
    np.full(df_sim_am.index.size, 8),
    np.full(df_bm.index.size, 30), 
    ])
axes = pd.plotting.scatter_matrix(df_merged, color=colors, s=sizes, alpha=1.0, diagonal=None, figsize=(10,10))
for i, dvar in enumerate(dvars):
    xmin, xmax = df_merged[rm_lab[dvar]].quantile([0.0, 0.998]).values
    for j in range(len(dvars)):
        axes[j,i].set_xlim([xmin, xmax])
        if j != i: 
            axes[i,j].set_ylim([xmin, xmax])
    # fix ticks ..
    xticks = axes[-1,i].get_xticks()
    xticks = xticks[np.logical_and(xticks>xmin, xticks<xmax)]
    axes[-1,i].set_xticks(xticks)
    axes[-1,i].set_xticklabels(xticks, rotation=0)
    if i == 0:
        ymin,ymax = axes[0,0].get_ylim()
        axes[i,i].set_yticks(xticks*(ymax-ymin)/(xmax-xmin))
        axes[i,i].set_yticklabels(xticks)
    
    # axes[i,i].lines.pop(0)
    # df_sim_am[dvar].plot.kde(ax=axes[i,i], color='k', label='sim', legend=i==0)
    # if dvar != 's':
    #     df_bm[dvar].plot.kde(ax=axes[i,i], color='r', label='obs', legend=i==0)
    # else:
    #     df_bm[dvar][:-2].plot.kde(ax=axes[i,i], color='r', label='obs', legend=i==0)
    # if i == 0:
    #     axes[i,i].set_ylabel(rm_lab[dvar])
    # else:
    #     axes[i,i].yaxis.set_visible(False)
    
for r in range(len(dvars)):
    for c in range(len(dvars)):
        if c >= r:
            axes[r,c].set_visible(False)
            continue

axes[1,0].plot(0,0, f'.r', label='sim')
axes[1,0].plot(0,0, f'.k', label='obs')
axes[1,0].legend()

plt.tight_layout()
plt.subplots_adjust(left=0.1,right=1,bottom=0.1,top=1, wspace=0, hspace=0)
# plt.savefig(join(r'../../4_results', 'stochastic_events_AM.png'), dpi=300, bbox_axes='tight')


## create forcing time series

In [ ]:
def sort_center(x, dim):
    a = x.get_axis_num(dim)
    n = x[dim].size
    x_sorted = np.apply_along_axis(np.sort, a, x)
    idx = [None if i!=a else range(n) for i in range(x.ndim)]
    reorder = np.append(np.arange(0, n, 2), np.arange(1, n, 2)[::-1])
    return xr.DataArray(np.take_along_axis(x_sorted, reorder[idx], a), x.coords)

In [ ]:
# discharge: qb, qp
# get design hydrographs by vertical averaging normalized AM peak hydrographs
df_rps.loc[0,:] = 0
df_rps = df_rps.sort_index()
q_event_lst = []
for dvar in ['qp', 'qb']:
    # update RP0 based on wet season average
    m = ds[dvar].time.dt.month
    df_rps.loc[0,dvar] = ds[dvar].isel(time=np.logical_or(m>=11, m<=4)).mean('time').compute().item()
    # create events
    daq_hydrograph = get_peak_hydrographs(ds[dvar], ds_peaks[dvar], wdw_size=21).compute().mean('peak')
    # if dvar in df_timelags:
    #     daq_hydrograph['time'] = daq_hydrograph['time'] + df_timelags[dvar]
    daq_events = df_rps[dvar].to_xarray() * daq_hydrograph
    q_event_lst.append(daq_events)
dsq_events = xr.merge(q_event_lst).dropna('time')
dsq_events['time'] = dsq_events['time']*24
dsq_events['time'].attrs.update(units='hour')
_ = dsq_events['qb'].sel(rps=100).plot.line(x='time')
_ = dsq_events['qp'].sel(rps=100).plot.line(x='time')
# dsq_events.to_netcdf(fnq.replace('.nc', '_events.nc'))

In [ ]:
# coastal

# update RPS h_tsw based on stochastic event set
df_sim_am0 = pd.read_csv('../../4_results/sim_AM_indep.csv', index_col=0)
dists['h_tsw'] = emperical_dist(df_sim_am0['h_tsw'].values, df_sim_am0['h_tsw'].size)
df_rps.loc[:,'h_tsw'] = dists['h_tsw'].ppf(1-1/df_rps.index.values)



In [ ]:
# get design hydrographs by horizontal averaging normalized AM peak hydrographs
dasw = dsh0['sw']#.rolling(time=48, center=True).mean('time')
dasw_peaks = get_peaks(dasw, "BM", min_dist=6*24*5, period='AS-AUG')
dasw_hydrographs = get_peak_hydrographs(
    dasw,
    dasw_peaks, 
    wdw_size=int(6*24*5), 
    normalize=True,
    # n_peaks=20
)
dasw_hydrographs = np.maximum(0,dasw_hydrographs.dropna('peak'))
dasw_hydrographs['time'] = dasw_hydrographs['time']/6  # hr
dasw_hydrograph = sort_center(dasw_hydrographs, dim='time').mean('peak') # horizontal averaging
# dasw_hydrograph = dasw_hydrographs.mean('peak') # vertical averaging

da_mhws_peaks = get_peaks(dsh0['t'], "BM", min_dist=6*24*10, period="29.5D")
da_mhws_hydrographs = get_peak_hydrographs(
    dsh0['t'], da_mhws_peaks, 
    wdw_size=int(6*24*14.5), 
    normalize=False,
)
da_mhws_hydrographs['time'] = da_mhws_hydrographs['time']/6 # hr
da_mhws_hydrograph = da_mhws_hydrographs.mean('peak')
mhws = 3.8 # highest astronomical tide IHO tidal constituents
df_rps.loc[0,'h_tsw'] = mhws
da_mhws_hydrograph = da_mhws_hydrograph/da_mhws_hydrograph.max('time')*mhws
dasw_hydrograph = dasw_hydrograph.reindex(time=da_mhws_hydrograph.time, fill_value=0)

da_h_events = da_mhws_hydrograph + dasw_hydrograph * (df_rps['h_tsw'].to_xarray()-mhws)
# da_h_events['time'] = da_h_events['time'] + df_timelags[['s','w']].min()*24
da_h_events['time'].attrs.update(units='hour')
_ = da_h_events.sel(rps=[0,2,500]).plot.line(x='time')
# da_h_events.to_netcdf(fnh.replace('.nc', '_events.nc'))


In [ ]:
# precip
# get design events from IDF curves and alternating block method
da_p_events = get_hyetograph(da_p_bm['return_values'], dt=1, length=durations[-1])
# da_p_events['time'] = da_p_events['time'] + df_timelags['p']*24
da_p_events['time'].attrs.update(units='hour')
_ = da_p_events.sel(rps=[2,500]).plot.line(x='time')
# da_p_events.to_netcdf(fnp.replace('.nc', '_events.nc'))

In [ ]:
df_sim_events.index.name = 'event'
df_sim_events1 = df_sim_events.reset_index().set_index(['event','year']).drop(columns=['t','s','w'])
for dvar in df_sim_events1.columns:
    thresh = dists[dvar].ppf(1-1/1.01)
    v = df_sim_events1[dvar]
    df_sim_events1[f'{dvar}_rp'] = np.minimum(500, np.where(v>thresh, 1/dists[dvar].sf(v), 0))
df_sim_events1.round(3).to_csv(r'../../4_results/sim_EVENTS_rp.csv')
df_sim_events1

In [ ]:
# get all scenarios used for linear interpolating damages
from itertools import product

def interpolator(coords1d, point) :
    dims = len(point)
    imax = len(coords1d)-1
    indices = []
    sub_coords = []
    for j in range(dims):
        idx = np.digitize([point[j]], coords1d)[0]
        indices += [[max(0,idx - 1), min(imax, idx)]]
        sub_coords += [coords1d[indices[-1]]]
    coords_out = np.array([j for j in product(*sub_coords)])
    return coords_out

rps = np.hstack([[0], _RPS])
rps = np.array([  0,   2,   5,  10,  25,  50, 100, 250, 500])
cols = ['qb_rp', 'qp_rp', 'p_rp', 'h_tsw_rp']
print(len(rps)**len(cols))
scens = []

values = df_sim_events1.loc[:,cols].values
for event in values:
    scens.append(interpolator(rps, event))
# include univariate
rps0 = np.zeros(4, dtype=int)
for i in range(4):
    for rp in rps[1:]:
        _rps = rps0.tolist()
        _rps[i] = rp
        scens.append(_rps)
# include full dependence
for rp in rps:
    scens.append(np.full(4, rp, dtype=int).tolist())
##
df_scen = pd.DataFrame(data=np.vstack(scens), columns=cols).value_counts().rename('count').reset_index()
df_scen['scen'] = [
    f"qb{qb_rp:03.0f}_qp{qp_rp:03.0f}_h{h_rp:03.0f}_p{p_rp:03.0f}" 
    for i, (qb_rp, qp_rp, p_rp, h_rp) in df_scen[cols].iterrows()
]
df_scen
print(df_scen.index.size)
df_scen.to_csv(r'../../4_results/sim_SCEN_indep.csv')
df_scen.sort_values('count', ascending=False).head(10)

In [ ]:
df_scen0 = pd.read_csv(r'../../4_results/sim_SCEN.csv', index_col=0)
df_scen[~np.isin(df_scen['scen'], df_scen0['scen'])]

In [ ]:
# df_rps.round(3).to_csv(r'../../4_results/rps.csv')
df_rps